# MOULIN'S WATER LEVEL TIMESCALE FLUCTUATIONS


    1.Reservoir-constriction equation from eq(5a)&(5b)
    2.Creep equation from eq(6) in Covington(2012)-> derived from Spring&Hutter(1981), 
    and also used in Arnold et al.(1998) model
    
![title](SketchMoulin.png)

    Constants:
    g = 9.81 m/s2
    
    Fixed variables:
    R 
    
    

In [35]:
%pylab
from scipy.integrate import odeint

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [47]:
# RESERVOIR-CONSTRICTION
########################

# Constant:
g = 9.81 #m/s2
Lf = 3.34*10.**5 # J/kg
n = 3. # Unitless, ice flow law exponent
B = 5.8*10.**7 # N/m^2*s, Arrhenius parameter
rhow = 1000. # kg/m^3, density of water
rhoi = 900. # kg/m^3, density of ice
f = 0.1 #unitless


def dh_Ac_dt(parameter, time, Recharge, Thickness, M_area, C_length):
    Fh,FAc = parameter
    D = sqrt(FAc*4./pi)
    #FAc = pi*(D**2)/4.
    Cf = 1. + f*Thickness/D
    Pwet = 2.*pi*D/2 # conduit wetted perimeter
    Q = FAc*sqrt(2.*g*Fh/Cf)
    Pw = rhow*g*Fh
    Pi = rhoi*g*Thickness    
    melt = (f*rhow*Pwet*Q**3.)/(8.*rhoi*Lf*FAc**3.)
    creep = 2.*(1./(n*B)**n)*FAc*(Pi-Pw)*abs(Pi-Pw)**(n-1.)
    return (Recharge-Q)/M_area, melt-creep





In [51]:
# head and conduit

for i in [1,2,3,4,5,7,10]:
    R = i #2. #m^3/s
    Z = 500. # m, thickness of the ice
    r = 10. # m, radius of the moulin
    Ar = pi*r**2. # m^2
    L = 1000. #m
    Days = 10     ; duration = Days*24*3600; tstep = duration/100; time = linspace(0,duration,tstep)
    initialpara=[320.,0.65]
    t = time/(24*3600) # from seconds to days

    result = odeint(dh_Ac_dt, initialpara, time, args=(R, Z, Ar, L))
    h = result[:,0]
    Ac = result[:,1]
    Dc = sqrt(Ac/pi)*2

    figure(figsize=(15,4))
    plot(t,h); title('Hydraulic head in the moulin'); ylabel('[m]'); xlabel('Day'); grid(); ylim([0,800])
    savefig('./Figure/Head-%d-%d-%d.pdf' % (R,Z,r))

    figure(figsize=(15,4))
    plot(t,Dc); title('Conduit Diameter'); ylabel('[m]'); xlabel('Day'); grid(); ylim([0,2])
    savefig('./Figure/Conduit-%d-%d-%d.pdf' % (R,Z,r))